In [1]:
import csv 
import pandas as pd
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import numpy as np
import nltk
import itertools
from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/kasem/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/kasem/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
def pre_process_df_keywords(df):
    for column in df.columns:
        for indx, entry in enumerate(df[column].dropna(axis=0)):
            if len(entry.split()) > 1 and entry[-1] != " ":
                df[column][indx] = entry + " "
def get_two_word(message):
    combined_2_words = []
    for word1, word2 in zip(message,list( message[1:] + list((message[0])))):
        combined_2_words.append(word1 + ' ' + word2)
    return combined_2_words

def pre_process(message):
    stopWords = set(stopwords.words('english'))
    pronouns = ["we", "We","WE","I","me","Me","ME","THEY","they","They","Them","them","THEM","YOU","You","you"]
    result = []
    for word in message.split():
        if word not in stopWords and word not in pronouns:
            result.append(word)
    return (result)
    
def gen_len(iter):
    return sum([1 for _ in iter])


def isMatch_1_word(word1,word2,thresh=80): return (fuzz.ratio(word1, word2) > thresh)
def isMatch_many_words(word1,message,thresh=90): return (fuzz.partial_ratio(word1, message) >= thresh)

def column_summary(word1, df):
    summary = np.zeros(len(df.columns))
    for col_indx, worker in enumerate(df.columns):
        for word2 in df[worker].dropna(axis=0):
            if isMatch_1_word(word1,word2):
                summary[col_indx]+=1
    return summary


def summarize_one_words(message, first):
    mat = np.zeros((len(message),len(first.columns)))
    for word_indx, word in enumerate(message):    
        mat[word_indx]=(column_summary(word,first)) 
    return mat

def summarize_two_words(message, df):
    mat = np.zeros((len(message),len(df.columns)))
    for col_indx, worker in enumerate(df.columns):
        two_words = df[df[worker].apply(lambda x: len(str(x).split())>1)][worker]
        for word in two_words.dropna(axis=0):
            for word_indx, two_word in enumerate(get_two_word(message)):
                if isMatch_many_words(word, two_word):
                    mat[word_indx][col_indx]+=1
    return mat

def find_workers(raw_message, message, first):
    mat = summarize_one_words(message, first) + summarize_two_words(message, first)
    #print(mat,"\n\n")
    for word_indx, row_word in enumerate(mat[:]):
        if row_word.sum() == 0:
            mat[word_indx] = np.zeros((len(first.columns))); continue
            
        if len(first.columns[row_word != 0]) > 1:
            mat[word_indx] = np.zeros((len(first.columns)))
    
    summary = np.array(mat).sum(axis=0)
    match_worker = first.columns[summary!=0]
    summary_wo_zeros = summary[np.argwhere(summary).reshape(1,-1)]
    
    print(mat)
    print(summary)
    print(f"""He is looking for >>> {match_worker}""")

    if summary_wo_zeros.max() != summary_wo_zeros.min():
        print(f"Highest match is: {first.columns[np.argmax(summary)]}")
    return match_worker

In [3]:
xls = pd.ExcelFile('strings stems.xlsx')
first = pd.read_excel(xls, 'first').dropna(axis=1, how='all').dropna(axis=0, how='all')
parent_second = pd.read_excel(xls, 'parent second').dropna(axis=1, how='all').dropna(axis=0, how='all')
bad_keywords= pd.read_excel(xls, 'bad keywords').dropna(axis=1, how='all').dropna(axis=0, how='all')
at_least_another= pd.read_excel(xls, 'at least another').dropna(axis=1, how='all').dropna(axis=0, how='all')

pre_process_df_keywords(first)
pre_process_df_keywords(parent_second)
pre_process_df_keywords(bad_keywords)

In [8]:
raw_message = """
I need business card and photoshop for color correction proof reader to draw images

""".lower()
print("Breaking") if step_1(raw_message, bad_keywords) else False
print(step_1(raw_message, bad_keywords))

message = pre_process(raw_message)
find_workers(raw_message, message, first)
pass

[]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[2. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
He is looking for >>> Index(['a graphic designer', 'someone', 'a proofreader'], dtype='object')
Highest match is: a graphic designer
